# Assignment 01: Data Preprocessing Project - Matthew Mendoza

I am to account for missing values and to concatenate rows and columns.

I will be using a dataset donated to UC Irvine Machine Learning Repository:
[Diabetes 130-US hospitals for years 1999-2008](https://archive.ics.uci.edu/dataset/296/diabetes+130-us+hospitals+for+years+1999-2008)

The dataset represents a ten-year (1999-2008) clinical record of diabetes
patients in 130 US hospitals, with the key goal of predicting early readmission
within 30 days of discharge to address inconsistent diabetes management, which
impacts both hospital costs and patient health.

## Precautions, cavitates, and concerns

Some "massaging" of the data is required...

`age` values assigned are not natural numbers, but are given a range with an
inclusive lower bound and an exclusive upper bound number in 10 year increments;
for example, `[0-10)`, `[10-20)`, `[20-30)`, ect.

`weight`, like `age`, is also a range of numbers with an inclusive lower bound
number and an exclusive upper bound number, but in 25 pounds increments
(e.g `[0-25`,`[50-75)`,`[100-125)`, ...).

It should be noted that `weight` in the dataset is not prioritized; in addition,
when there are records for `weight` only `Caucasian`s are those who were
accounted for (excluding `Asian`, `African American`, `Hispanic`, and `other`)

For this reason I am forced to list the median weights of each category,
regardless if they were male or female, of only `Caucasian` in the dataset.

No "real" analysis or extrapolation can be derived form the data produced.
This is purely an exercise and demonstration of how to:

- Drop any rows with any NA values
- Replace missing values with the median value for that column
- Rows and columns can be concatenated together to form new data frames

## Dataset citation

Clore,John, Cios,Krzysztof, DeShazo,Jon, and Strack,Beata. (2014). Diabetes 130-US hospitals for years 1999-2008. UCI Machine Learning Repository. https://doi.org/10.24432/C5230J.

## [Python](https://www.python.org/) dependencies

For analysis and processing of data I will be using the
[pandas package](https://pypi.org/project/pandas/)


## Missing values

In this dataset there are missing values indicated by a question mark (`?`).
The missing values that are missing are categorical in type and include the
following:

- `race`: Caucasian, Asian, African American, Hispanic, and other
- `weight`: Weight in pounds
- `payer_code`: Integer identifier corresponding to 23 distinct values,
  for example, Blue Cross/Blue Shield, Medicare, and self-pay
- `medical_specialty`: Integer identifier of a specialty of the admitting
  physician, corresponding to 84 distinct values, for example, cardiology,
  internal medicine, family/general practice, and surgeon
- `diag_1`: The primary diagnosis (coded as first three digits of ICD9);
  848 distinct values
- `diag_2`: Secondary diagnosis (coded as first three digits of ICD9);
  923 distinct values
- `diag_3`: Additional secondary diagnosis (coded as first three digits of ICD9);
  954 distinct values


In [ ]:
import pandas as pd
import os
import numpy as np

# Set the path to the data
DATA_PATH: str = os.getcwd() + \
    os.path.normpath(
        path="/data/diabetes+130-us+hospitals+for+years+1999-2008")

# Construct file paths
input_file = os.path.join(DATA_PATH, "diabetic_data.csv")
output_file = os.path.join(
    DATA_PATH, "diabetic_data_caucasian_median_weight.csv")

# Load the original CSV data
data = pd.read_csv(input_file, na_values='?')

# Filter the data to include only Caucasian patients
caucasian_data = data[data['race'] == 'Caucasian']


def calculate_median_weight(weight_range: str) -> float:
    """Calculate the median weight from a weight range or single value.

    Args:
        weight_range (str): A string containing a weight range or single value.

    Returns:
        float: The median weight calculated from the given weight range or value.
    """
    if isinstance(weight_range, str):
        bounds: list = [int(val) for val in weight_range.strip(
            '[]()').split('-') if val.isdigit()]
        if len(bounds) == 2:
            # If the value is a range, return the median
            return np.median(bounds)
    return np.nan  # Handle non-numeric values and floats as NaN


# Calculate the median for the 'weight' column
median = caucasian_data['weight'].apply(calculate_median_weight)

# Create a new DataFrame with the desired columns
output_data = caucasian_data[['encounter_id',
                              'patient_nbr', 'race', 'gender', 'age', 'weight']]

# Replace "NaN" values in the 'weight' column with the calculated median
output_data.loc[:, 'weight'] = output_data['weight'].apply(
    calculate_median_weight).fillna(median)

# Drop rows with missing values (NaN) in any column and update the DataFrame in-place
output_data.dropna(inplace=True)

# Save the filtered data to a new CSV file
output_data.to_csv(output_file, index=False)

In [123]:
pd.read_csv(input_file) # Original data

,encounter_id,patient_nbr,race,gender,age,weight,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,...,citoglipton,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted
0,2278392,8222157,Caucasian,Female,[0-10),?,6,25,1,1,...,No,No,No,No,No,No,No,No,No,NO
1,149190,55629189,Caucasian,Female,[10-20),?,1,1,7,3,...,No,Up,No,No,No,No,No,Ch,Yes,>30
2,64410,86047875,AfricanAmerican,Female,[20-30),?,1,1,7,2,...,No,No,No,No,No,No,No,No,Yes,NO
3,500364,82442376,Caucasian,Male,[30-40),?,1,1,7,2,...,No,Up,No,No,No,No,No,Ch,Yes,NO
4,16680,42519267,Caucasian,Male,[40-50),?,1,1,7,1,...,No,Steady,No,No,No,No,No,Ch,Yes,NO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101761,443847548,100162476,AfricanAmerican,Male,[70-80),?,1,3,7,3,...,No,Down,No,No,No,No,No,Ch,Yes,>30
101762,443847782,74694222,AfricanAmerican,Female,[80-90),?,1,4,5,5,...,No,Steady,No,No,No,No,No,No,Yes,NO
101763,443854148,41088789,Caucasian,Male,[70-80),?,1,1,7,1,...,No,Down,No,No,No,No,No,Ch,Yes,NO
101764,443857166,31693671,Caucasian,Female,[80-90),?,2,3,7,10,...,No,Up,No,No,No,No,No,Ch,Yes,NO


In [120]:
data # Original data (but with NaN values) - for comparison

,encounter_id,patient_nbr,race,gender,age,weight,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,...,citoglipton,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted
0,2278392,8222157,Caucasian,Female,[0-10),NaN,6,25,1,1,...,No,No,No,No,No,No,No,No,No,NO
1,149190,55629189,Caucasian,Female,[10-20),NaN,1,1,7,3,...,No,Up,No,No,No,No,No,Ch,Yes,>30
2,64410,86047875,AfricanAmerican,Female,[20-30),NaN,1,1,7,2,...,No,No,No,No,No,No,No,No,Yes,NO
3,500364,82442376,Caucasian,Male,[30-40),NaN,1,1,7,2,...,No,Up,No,No,No,No,No,Ch,Yes,NO
4,16680,42519267,Caucasian,Male,[40-50),NaN,1,1,7,1,...,No,Steady,No,No,No,No,No,Ch,Yes,NO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101761,443847548,100162476,AfricanAmerican,Male,[70-80),NaN,1,3,7,3,...,No,Down,No,No,No,No,No,Ch,Yes,>30
101762,443847782,74694222,AfricanAmerican,Female,[80-90),NaN,1,4,5,5,...,No,Steady,No,No,No,No,No,No,Yes,NO
101763,443854148,41088789,Caucasian,Male,[70-80),NaN,1,1,7,1,...,No,Down,No,No,No,No,No,Ch,Yes,NO
101764,443857166,31693671,Caucasian,Female,[80-90),NaN,2,3,7,10,...,No,Up,No,No,No,No,No,Ch,Yes,NO


In [122]:
output_data # Filtered data (showing only Caucasian patients with median weight)

,encounter_id,patient_nbr,race,gender,age,weight
1102,8042082,94466574,Caucasian,Male,[50-60),87.5
1132,8235036,79874631,Caucasian,Female,[60-70),12.5
1136,8274126,90421380,Caucasian,Female,[40-50),87.5
1190,8421486,55628172,Caucasian,Female,[70-80),87.5
1204,8576406,80041266,Caucasian,Female,[80-90),62.5
...,...,...,...,...,...,...
101449,439577312,97708986,Caucasian,Female,[70-80),62.5
101516,440252540,97508430,Caucasian,Male,[60-70),112.5
101640,442077494,62420742,Caucasian,Female,[70-80),62.5
101667,442577684,78208677,Caucasian,Male,[70-80),87.5


## Concatenating rows and columns

Rows and columns can be concatenated together to form new data frames.


In [ ]:
# Create a new dataframe from patient number and weight

import os
import pandas as pd

# Set the path to the data
DATA_PATH: str = os.getcwd() + \
    os.path.normpath(
        path="/data/diabetes+130-us+hospitals+for+years+1999-2008")

# Construct file paths
input_file = os.path.join(
    DATA_PATH, "diabetic_data_caucasian_median_weight.csv")
df = pd.read_csv(input_file, na_values=['NA', '?'])

column_patient_nbr = df['patient_nbr']
column_weight = df['weight']

# axis=1 for columns and axis=0 for rows
df_new = pd.concat([column_patient_nbr, column_weight], axis=1)

df_new # New dataframe with patient number and weight only